In [ ]:
import gradio as gr
import os
import pandas as pd
import json
from PIL import Image, ImageSequence
import io
from functools import reduce

# Load the JSON file with rank data
with open("rank_data_03_25_2025.json", "r") as f:
    rank_data = json.load(f)

#######################################################
# Helper functions to build individual game leaderboards
#######################################################

def get_mario_leaderboard():
    data = rank_data.get("Super Mario Bros", {}).get("results", [])
    df = pd.DataFrame(data)
    df = df.rename(columns={
        "model": "Player", 
        "progress": "Progress (current/total)", 
        "score": "Score", 
        "time_s": "Time (s)"
    })
    return df

def get_2048_leaderboard():
    data = rank_data.get("2048", {}).get("results", [])
    df = pd.DataFrame(data)
    df = df.rename(columns={
        "model": "Player", 
        "score": "Score", 
        "steps": "Steps", 
        "time": "Time"
    })
    return df

def get_tetris_leaderboard():
    data = rank_data.get("Tetris (complete)", {}).get("results", [])
    df = pd.DataFrame(data)
    df = df.rename(columns={
        "model": "Player", 
        "score": "Score", 
        "steps_blocks": "Steps"
    })
    return df

def get_tetris_planning_leaderboard():
    data = rank_data.get("Tetris (planning only)", {}).get("results", [])
    df = pd.DataFrame(data)
    df = df.rename(columns={
        "model": "Player", 
        "score": "Score", 
        "steps_blocks": "Steps"
    })
    return df

def get_candy_leaderboard():
    data = rank_data.get("Candy Crash", {}).get("results", [])
    df = pd.DataFrame(data)
    df = df.rename(columns={
        "model": "Player", 
        "score_runs": "Score Runs", 
        "average_score": "Average Score", 
        "steps": "Steps"
    })
    return df

def get_sokoban_leaderboard():
    data = rank_data.get("Sokoban", {}).get("results", [])
    df = pd.DataFrame(data)
    df = df.rename(columns={
        "model": "Player", 
        "levels_cracked": "Levels Cracked", 
        "steps": "Steps"
    })
    return df

#######################################################
# Combined leaderboard: one row per model with one score column per game
#######################################################

def get_combined_leaderboard(selected_games):
    dfs = []
    # For each game, if its overall checkbox is True, extract its aggregated scoreboard
    if selected_games.get("Mario"):
        df_mario = get_mario_leaderboard()[["Player", "Score"]].copy()
        df_mario = df_mario.rename(columns={"Score": "Mario Score"})
        dfs.append(df_mario)
    if selected_games.get("2048"):
        df_2048 = get_2048_leaderboard()[["Player", "Score"]].copy()
        df_2048 = df_2048.rename(columns={"Score": "2048 Score"})
        dfs.append(df_2048)
    if selected_games.get("Tetris (complete)"):
        df_tetris = get_tetris_leaderboard()[["Player", "Score"]].copy()
        df_tetris = df_tetris.rename(columns={"Score": "Tetris (complete) Score"})
        dfs.append(df_tetris)
    if selected_games.get("Tetris (planning)"):
        df_tetris_plan = get_tetris_planning_leaderboard()[["Player", "Score"]].copy()
        df_tetris_plan = df_tetris_plan.rename(columns={"Score": "Tetris (planning) Score"})
        dfs.append(df_tetris_plan)
    if selected_games.get("Candy Crash"):
        df_candy = get_candy_leaderboard()[["Player", "Average Score"]].copy()
        df_candy = df_candy.rename(columns={"Average Score": "Candy Crash Score"})
        dfs.append(df_candy)
    if selected_games.get("Sokoban"):
        df_sokoban = get_sokoban_leaderboard()[["Player", "Levels Cracked"]].copy()
        def max_level(s):
            try:
                parts = s.replace(',', ';').split(';')
                nums = [int(x.strip()) for x in parts if x.strip().isdigit()]
                return max(nums) if nums else "_"
            except:
                return "_"
        df_sokoban["Sokoban Score"] = df_sokoban["Levels Cracked"].apply(max_level)
        df_sokoban = df_sokoban[["Player", "Sokoban Score"]]
        dfs.append(df_sokoban)
    if dfs:
        df_combined = reduce(lambda left, right: pd.merge(left, right, on="Player", how="outer"), dfs)
    else:
        df_combined = pd.DataFrame()
    df_combined = df_combined.fillna("_")
    return df_combined

AttributeError: 'Dataframe' object has no attribute 'update'